# Load Data, Sanity Checks

In [1]:
import pandas as pd
import numpy as np

In [2]:
!ls

Patient Labelling - GPT-Labled-Transcript.csv
Patient Labelling - Manual-Labled-Transcript.csv
Patient2_Scoring.ipynb


In [3]:
labelled_transcript = pd.read_csv("/Users/djeong/Documents/2024_Harvard/Spring/MIT6.7930/ML4HC_RubricScoring/Presence Labelling/data/20/transcript_gpt.csv")
labelled_note = pd.read_csv("/Users/djeong/Documents/2024_Harvard/Spring/MIT6.7930/ML4HC_RubricScoring/Presence Labelling/data/20/note_gpt.csv")





In [4]:
labelled_transcript.head()

,Timestamp,Script,Placenta Previa,Placenta Accreta Spectrum or concern,Multiple Gestation,Polyhydramnios,Large for Gestational Age,Fibroids,Preeclampsia,HELLP,Anticoagulation use,Baseline Laboratory Values - Hemoglobin/Hematocrit,Baseline Laboratory Values - Platelet Count,"Baseline Laboratory Values - Coagulopathy (PTT, PT/INR)"
0,00:00.000 --> 00:07.000,Hello. Good morning. This is Dr. Smith from th...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00:07.000 --> 00:19.000,I'm looking for Ms. Jane Blank. This is she. H...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00:19.000 --> 00:24.000,Do you have 10 to 15 minutes to speak with us?...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00:24.000 --> 00:29.000,We've reviewed your chart and we'd just like t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00:29.000 --> 00:44.000,I know that in the past you had a history of a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
labelled_note.head()

,Script,Placenta Previa,Placenta Accreta Spectrum or concern,Multiple Gestation,Polyhydramnios,Large for Gestational Age,Fibroids,Preeclampsia,HELLP,Anticoagulation use,Baseline Laboratory Values - Hemoglobin/Hematocrit,Baseline Laboratory Values - Platelet Count,"Baseline Laboratory Values - Coagulopathy (PTT, PT/INR)"
0,High-Risk Anesthesia Consult Note,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Patient: Jane Blank,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Age: 32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MRN: 000000019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gravida/Para: G3P1011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# get rid of whitespace around text

labelled_transcript.columns = labelled_transcript.columns.str.strip()
labelled_transcript = labelled_transcript.applymap(lambda x: x.strip() if isinstance(x, str) else x)

labelled_transcript.replace('', np.nan, inplace=True)


labelled_note.columns = labelled_note.columns.str.strip()
labelled_note = labelled_note.applymap(lambda x: x.strip() if isinstance(x, str) else x)

labelled_note.replace('', np.nan, inplace=True)


/var/folders/c5/k58v_7fn31j_jp5cz2ggmbs00000gn/T/ipykernel_48148/426885368.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  labelled_transcript = labelled_transcript.applymap(lambda x: x.strip() if isinstance(x, str) else x)
/var/folders/c5/k58v_7fn31j_jp5cz2ggmbs00000gn/T/ipykernel_48148/426885368.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  labelled_note = labelled_note.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [7]:
(set(labelled_transcript.columns) - set(["Sentences"])) == (set(labelled_note.columns) - set(["Sentences"]))

False

In [8]:
labelled_note.iloc[10]

Script                                                     Ms. Jane Blank is a 32-year-old G3P1011 at 33w...
Placenta Previa                                                                                     Positive
Placenta Accreta Spectrum or concern                                                                Positive
Multiple Gestation                                                                                       NaN
Polyhydramnios                                                                                           NaN
Large for Gestational Age                                                                                NaN
Fibroids                                                                                                 NaN
Preeclampsia                                                                                             NaN
HELLP                                                                                                    NaN
Anticoagulation use

# Per feature comparison

In [9]:
def collectSentences(df, ref, feature_i, value):
    # Step 1: Get rows where feature_i equals the target value
    if value == 'N':
        match_values = {'negative', 'Negative', 'n', 'N'}
    elif value == 'P':
        match_values = {'positive', 'Positive', 'p', 'P'}
    else:
        match_values = {value}

    # Filter rows where feature_i is in the set of match_values
    matches = df[df[feature_i].isin(match_values)]

    # Step 2: Create list of (index, sentence)
    result = [(idx, row[ref]) for idx, row in matches.iterrows()]

    # Step 3: Merge adjacent entries
    merged = []
    start_idx, current_idx, current_text = result[0][0], result[0][0], result[0][1]

    for next_idx, next_text in result[1:]:
        if next_idx == current_idx + 1:
            # Continue current group
            current_text += " " + next_text
            current_idx = next_idx
        else:
            # End current group and start new
            merged.append(((start_idx, current_idx), current_text))
            start_idx = current_idx = next_idx
            current_text = next_text

    # Add final group
    merged.append(((start_idx, current_idx), current_text))

    return merged


In [10]:
labelled_note.iloc[10]

Script                                                     Ms. Jane Blank is a 32-year-old G3P1011 at 33w...
Placenta Previa                                                                                     Positive
Placenta Accreta Spectrum or concern                                                                Positive
Multiple Gestation                                                                                       NaN
Polyhydramnios                                                                                           NaN
Large for Gestational Age                                                                                NaN
Fibroids                                                                                                 NaN
Preeclampsia                                                                                             NaN
HELLP                                                                                                    NaN
Anticoagulation use

In [11]:
import emoji 

# Variables for completeness and curation scores

features_in_transcript = 0
correct_features_in_note = 0

features_not_in_transcript = 0
correct_features_not_in_note = 0

# Iterate through features
for feature_i in (set(labelled_transcript.columns) - set([
    "Script",
    "Timestamp",
    "Sentence",
    "Baseline Laboratory Values - Hemoglobin/Hematocrit",
    "Baseline Laboratory Values - Platelet Count",
    "Baseline Laboratory Values - Coagulopathy (PTT, PT/INR)"
])):
# for feature_i in ["Placenta Accreta Spectrum or concern"]:
    print("\n<<<<<<\n")
    print(feature_i)
    # value counts
    vc_transcript = labelled_transcript[feature_i].value_counts().to_dict() if labelled_transcript[feature_i].notna().any() else {'M' : -1}
    vc_note = labelled_note[feature_i].value_counts().to_dict() if labelled_note[feature_i].notna().any() else {'M' : -1}

    print(vc_transcript)
    
    def relabel(counts):
        if len(counts)==0:
            return 'Missing'
        
        max_count = max(counts.values())
        most_frequent = [k for k, v in counts.items() if v == max_count]
        print("MOST FREQUENT", most_frequent)
        assert len(most_frequent) == 1
        return {most_frequent[0] : -1}

    vc_transcript, vc_note = relabel(vc_transcript), relabel(vc_note)
    print(vc_transcript)
    
    # check that each risk factor only has one label
    assert len(vc_transcript.keys())==1 and len(vc_note.keys())==1, f"Labels are not size 1 {vc_transcript.keys()} {vc_note.keys()}"

    # check that values are valid
    valid_keys = {"Positive", "Negative", "M"}
    assert set(vc_transcript.keys()).issubset(valid_keys), f"Invalid keys found! {vc_transcript}"
    assert set(vc_note.keys()).issubset(valid_keys), "Invalid keys found!"

    # map keys to one-letter

    mapOneLetter = {'Positive' : 'P', 'Negative' : 'N', 'M' : 'M'}
    vc_transcript = { mapOneLetter[k] : v for k, v in vc_transcript.items()}
    vc_note = { mapOneLetter[k] : v for k, v in vc_note.items()}

    print("VC_TRANSCRIPT", vc_transcript, "VC_NOTE", vc_note) 
    
    # collect counts for completeness and curation

    if 'M' in vc_transcript:
        features_not_in_transcript += 1

        if 'M' in vc_note:
            correct_features_not_in_note += 1
        else:
            pass
            # print("WRONG1", feature_i, vc_transcript, vc_note)
    else:
        features_in_transcript += 1

        if 'M' not in vc_note:
            correct_features_in_note += 1
        else:
            pass
             # print("WRONG2", feature_i, vc_transcript, vc_note)
    
    # If both are missing, there are no problems
    if list(vc_transcript.keys())==['M'] and list(vc_note.keys())==['M']:
        print(emoji.emojize(':thumbs_up:'), f"{feature_i} is M in both transcript and note")
        continue 

    # compare (for loop just for ease of implementation)
    for key_transcript in vc_transcript:
        for key_note in vc_note:
            if key_transcript == key_note: # no problems
                print(emoji.emojize(':thumbs_up:'), f"{feature_i} is {key_transcript} in both transcript and note")
            elif (key_transcript, key_note) in [
                ('P', 'N'),
                ('P', 'M'), 
                ('N', 'P'),
                ('N', 'M')
            ]: # knowledge gap error
                print(f"{feature_i} has a knowledge gap error")

                print(f"The transcript says that the presence of {feature_i} is {'Positive' if key_transcript=='P' else 'Negative'}")
                print(collectSentences(labelled_transcript, 'Script', feature_i, key_transcript))

                if key_note == 'M':
                    print(f"The note does not mention the presence of {feature_i}")
                else:
                    print(f"The note says that the presence of {feature_i} is {'Positive' if key_note=='P' else 'Negative'}")
                    print(collectSentences(labelled_note, 'Script', feature_i, key_note))
                    
            elif (key_transcript, key_note) in [
                ('M', 'P'),
                ('M', 'N')
            ]: # hallucination error
                print(f"{feature_i} has a hallucination error")

                print(f"The transcript does not mention the presence of {feature_i}")

                print(f"The note says that the presence of {feature_i} is {'Positive' if key_note=='P' else 'Negative'}")
                print(collectSentences(labelled_note, 'Script', feature_i, key_note))
                
            else: # should never reach this point
                print(emoji.emojize(':angry_face_with_horns:'), "PROBLEM IN CODE")


<<<<<<

HELLP
{'M': -1}
MOST FREQUENT ['M']
MOST FREQUENT ['M']
{'M': -1}
VC_TRANSCRIPT {'M': -1} VC_NOTE {'M': -1}
👍 HELLP is M in both transcript and note

<<<<<<

Anticoagulation use
{'Negative': 2}
MOST FREQUENT ['Negative']
MOST FREQUENT ['Negative']
{'Negative': -1}
VC_TRANSCRIPT {'N': -1} VC_NOTE {'N': -1}
👍 Anticoagulation use is N in both transcript and note

<<<<<<

Preeclampsia
{'Negative': 2}
MOST FREQUENT ['Negative']
MOST FREQUENT ['M']
{'Negative': -1}
VC_TRANSCRIPT {'N': -1} VC_NOTE {'M': -1}
Preeclampsia has a knowledge gap error
The transcript says that the presence of Preeclampsia is Negative
[((15, 16), 'Okay. Any preeclampsia or high blood pressure in pregnancy? No.')]
The note does not mention the presence of Preeclampsia

<<<<<<

Placenta Accreta Spectrum or concern
{'Positive': 3}
MOST FREQUENT ['Positive']
MOST FREQUENT ['Positive']
{'Positive': -1}
VC_TRANSCRIPT {'P': -1} VC_NOTE {'P': -1}
👍 Placenta Accreta Spectrum or concern is P in both transcript and not

In [12]:
print("Completeness Score/Sensitivity:", correct_features_in_note/features_in_transcript)
print("Curation Score/Specificity:", correct_features_not_in_note/features_not_in_transcript)


Completeness Score/Sensitivity: 0.6
Curation Score/Specificity: 0.5
